In [2]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
 
default = {'scenario': 'base','training_folder':'../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

In [3]:
import os
import time
import geopandas as gpd
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
from syspy.skims.skims import euclidean
from syspy.syspy_utils.pandas_utils import groupby_weighted_average
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [4]:
scenario = argv['scenario']


on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)

training_folder = argv['training_folder']
input_folder = training_folder + r'/inputs/'

if not on_lambda:
    scenario_folder = training_folder + '/scenarios/' + scenario + '/inputs/'
    output_folder = scenario_folder + 'calibration/'
    model_folder = training_folder + '/scenarios/' + scenario + '/model/'
    calib_folder = input_folder + scenario    + '/calibration/'
else:
    scenario_folder = input_folder
    output_folder = scenario_folder + '/calibration/'
    model_folder = training_folder + '/model/'
    calib_folder = input_folder + scenario + '/calibration/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)
print('calib folder : ', calib_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/base/inputs/calibration/
scen folder :  ../../scenarios/base/inputs/
model folder :  ../../scenarios/base/model/
calib folder :  ../../inputs/base/calibration/


In [5]:
try:
            os.makedirs(output_folder)
except FileExistsError:
            pass

# Import data

In [6]:
def ratio_par_zones(self):
    df = self.copy()
    df["ratio"] = df["population"] / df["population"].sum()
    
    return(df[["ratio", 'NAME']])

In [13]:
road_count = pd.read_excel(calib_folder + 'comptages.xlsx', index_col='Link Index')

In [14]:
road_count

,Car_Dveh,PMV_Dveh,Truck_Dveh,Note
Link Index,,,,
rlink_7393,7342,2103,414,Waigani Market_SB
rlink_3602,6858,1942,283,Waigani Market_NB
rlink_7620,10727,3223,770,Waigani Tunnel_SB
rlink_831,9233,2955,629,Waigani Tunnel_NB
rlink_7711,8824,3436,720,Waigani Tunnel 2_SB
rlink_10810,8382,3236,505,Waigani Tunnel 2_NB
rlink_11134,10726,1622,1035,Waigani Freeway_SB
rlink_4764,10409,1849,1139,Waigani Freeway_NB
rlink_11782,8373,1113,752,Sunny Bunny_SB


In [9]:
road_count = road_count.rename_axis('index')

In [11]:
road_count = road_count.rename(columns={'Car_Dveh': 'car', 'PMV_Dveh':'pmv', 'Truck_Dveh':'truck'})

,car,pmv,truck,Note
index,,,,
rlink_7393,7342,2103,414,Waigani Market_SB
rlink_3602,6858,1942,283,Waigani Market_NB
rlink_7620,10727,3223,770,Waigani Tunnel_SB
rlink_831,9233,2955,629,Waigani Tunnel_NB
rlink_7711,8824,3436,720,Waigani Tunnel 2_SB
rlink_10810,8382,3236,505,Waigani Tunnel 2_NB
rlink_11134,10726,1622,1035,Waigani Freeway_SB
rlink_4764,10409,1849,1139,Waigani Freeway_NB
rlink_11782,8373,1113,752,Sunny Bunny_SB


In [12]:
#road_count.to_csv(output_folder +r'road_counts.csv')

In [6]:
od_car = pd.read_excel(calib_folder + 'od_car.xlsx', index_col=0)
od_car = pd.DataFrame(od_car.stack())
od_car.rename(columns={0: 'volume'}, inplace=True)
od_car.rename_axis(['origin', 'destination'], inplace=True)

od_pt = pd.read_excel(calib_folder + 'od_pt.xlsx', index_col=0)
od_pt = pd.DataFrame(od_pt.stack())
od_pt.rename(columns={0: 'volume'}, inplace=True)
od_pt.rename_axis(['origin', 'destination'], inplace=True)

In [7]:
od_walk = pd.read_excel(calib_folder + 'od_walk.xlsx', index_col=0)
od_walk = pd.DataFrame(od_walk.stack())
od_walk.rename(columns={0: 'volume'}, inplace=True)
od_walk.rename_axis(['origin', 'destination'], inplace=True)

In [8]:
od = pd.read_csv(calib_folder+'od.csv', index_col=0)
od

,origin,destination,volume_pt,volume_car,volume_walk,volumes_calibration
0,zone_bc_1,zone_bc_1,99.0,1092.0,0.0,1191.0
1,zone_bc_1,zone_bc_3,60.0,60.0,0.0,120.0
2,zone_bc_1,zone_bc_4,52.0,0.0,0.0,52.0
3,zone_bc_1,zone_bc_9,180.0,53.0,0.0,233.0
4,zone_bc_1,zone_bc_11,507.0,239.0,0.0,746.0
...,...,...,...,...,...,...
373,zone_bc_19,zone_bc_19,0.0,0.0,610.0,610.0
374,zone_bc_23,zone_bc_17,0.0,0.0,78.0,78.0
375,zone_bc_24,zone_bc_40,0.0,0.0,155.0,155.0
376,zone_bc_24,zone_bc_41,0.0,0.0,88.0,88.0


In [11]:
emissions= od.groupby('origin')['volumes_calibration'].sum()
attraction = od.groupby('destination')['volumes_calibration'].sum()

In [129]:
part_modale_calibration = od.groupby('origin')[['volume_car', 'volume_walk', 'volume_pt', 'volumes_calibration']].sum()

In [131]:
part_modale_calibration['part_car'] = part_modale_calibration['volume_car']/part_modale_calibration['volumes_calibration']
part_modale_calibration['part_pt'] = part_modale_calibration['volume_pt']/part_modale_calibration['volumes_calibration']
part_modale_calibration['part_walk'] = part_modale_calibration['volume_walk']/part_modale_calibration['volumes_calibration']

In [148]:
#part_modale_calibration.to_csv(output_folder +r'part_modale_calibration.csv')

In [135]:
part_modale_calibration.reset_index(inplace=True)

In [147]:
part_modale_calibration

,index,volume_car,volume_walk,volume_pt,volumes_calibration,part_car,part_pt,part_walk
0,zone_bc_1,5252.0,0.0,3194.0,8446.0,0.621833,0.378167,0.000000
1,zone_bc_10,1072.0,0.0,1034.0,2106.0,0.509022,0.490978,0.000000
2,zone_bc_11,23380.0,18103.0,43820.0,85303.0,0.274082,0.513698,0.212220
3,zone_bc_13,26261.0,25933.0,84795.0,136989.0,0.191702,0.618991,0.189307
4,zone_bc_14,6039.0,18853.0,41574.0,66466.0,0.090858,0.625493,0.283649
5,zone_bc_15,696.0,0.0,364.0,1060.0,0.656604,0.343396,0.000000
6,zone_bc_16,25002.0,23823.0,42130.0,90955.0,0.274883,0.463196,0.261921
7,zone_bc_17,680.0,629.0,1323.0,2632.0,0.258359,0.502660,0.238982
8,zone_bc_19,1636.0,1093.0,6585.0,9314.0,0.175650,0.707000,0.117350
9,zone_bc_20,3940.0,1192.0,19249.0,24381.0,0.161601,0.789508,0.048891


In [146]:
part_modale_calibration.rename(columns={'origin':'index'}, inplace=True)

In [39]:
od = pd.read_csv(calib_folder + 'OD.csv')
all_columns = pd.read_csv(calib_folder + 'all_columns.csv')
blocks_fix = gpd.read_file(calib_folder + 'blocks_fix.geojson')
part_modale = gpd.read_file(calib_folder + 'part_modale.geojson')

In [40]:
zones = gpd.read_file(scenario_folder  + 'zones.geojson',driver='GeoJSON').set_index("index")
zones = (zones.groupby('NAME').apply(ratio_par_zones))#.droplevel(1)
zones['index'] = zones.index

In [41]:
#find ratio PPAM/ALL_PERIOD
all_columns['PPAM'] = all_columns[['S0801_C01_030E', 'S0801_C01_031E', 'S0801_C01_032E', 'S0801_C01_033E', 'S0801_C01_034E', 'S0801_C01_035E']].astype(float).sum(axis=1)
all_columns ['ALL_PERIOD'] = all_columns[['S0801_C02_027E','S0801_C02_028E','S0801_C02_029E', 'S0801_C01_030E', 'S0801_C01_031E', 'S0801_C01_032E', 'S0801_C01_033E', 'S0801_C01_034E', 'S0801_C01_035E', 'S0801_C02_036E']].astype(float).sum(axis=1)
all_columns['GEOID'] = all_columns['GEO_ID'].apply(lambda x: x[9:]).astype(str)

## Data preparation Calibration GENERATION/DISTRIBUTION

In [42]:
# import des OD totales
od['volumes'] = od[['S000', 'SA01', 'SA02', 'SA03', 'SE01','SE02', 'SE03', 'SI01', 'SI02', 'SI03']].sum(axis = 1)

In [43]:
part_modale = part_modale.loc[~part_modale['Geography'].isna()]
part_modale['County'] = part_modale['Geography'].apply(lambda x: x[11:14])
part_modale['TRACTCE20'] = part_modale['Geography'].apply(lambda x: x[-6:])
part_modale['car'] = part_modale['Estimate_Car, truck, or van']
part_modale['pt'] = part_modale['Estimate_Public transportation (excluding taxicab)']
part_modale['walk'] = part_modale['Estimate_Walked'] + part_modale['Estimate_Bicycle']

block_to_tract = blocks_fix[['GEOID20', 'TRACTCE20']]
block_name_tract = part_modale[['TRACTCE20', 'NAME','car','pt','walk']].merge(block_to_tract, on = 'TRACTCE20',  how='right')
block_name_tract = block_name_tract.drop_duplicates(['NAME', 'GEOID20'])



In [44]:
block_name_tract['GEOID20_2'] = block_name_tract['GEOID20'].astype(str).apply(lambda x : x[0:11]).astype(str)
block_name_tract = block_name_tract.merge(all_columns[['GEOID', 'PPAM', 'ALL_PERIOD']], left_on= 'GEOID20_2', right_on='GEOID', how='left')
block_name_tract = block_name_tract.drop(['GEOID20_2','GEOID'], axis=1)
block_name_tract['part_PPAM'] = block_name_tract['PPAM']/block_name_tract['ALL_PERIOD']


In [45]:
vol_od_calibration = od.merge(block_name_tract[['NAME', 'GEOID20','part_PPAM']], left_on= 'h_geocode', right_on= 'GEOID20', how = 'left')
vol_od_calibration = vol_od_calibration.merge(block_name_tract[['NAME', 'GEOID20']],left_on= 'w_geocode', right_on= 'GEOID20', how = 'left', suffixes= ('_origin', '_destination'))
vol_od_calibration = vol_od_calibration.loc[~ vol_od_calibration['NAME_destination'].isna()][['NAME_origin', 'NAME_destination', 'volumes', 'part_PPAM']]
vol_od_calibration = vol_od_calibration.groupby(['NAME_origin', 'NAME_destination']).agg({'volumes': 'sum', 'part_PPAM': 'first'}).reset_index()
vol_od_calibration['volumes_PPAM'] = vol_od_calibration['part_PPAM'] * vol_od_calibration['volumes']

vol_od_calibration = vol_od_calibration[["NAME_origin","NAME_destination","volumes_PPAM"]].merge(zones[['ratio','index', 'NAME']], left_on=  'NAME_origin', right_on= 'NAME', how = 'left')
vol_od_calibration = vol_od_calibration.merge(zones[['index', 'NAME', 'ratio']], left_on=  'NAME_destination', right_on= 'NAME', how = 'left' , suffixes = ('_origin', '_destination'))
vol_od_calibration["volumes_PPAM"] = vol_od_calibration['volumes_PPAM'] * (vol_od_calibration['ratio_origin'] * vol_od_calibration['ratio_destination'])
vol_od_calibration = vol_od_calibration.groupby(['index_origin', 'index_destination' ], as_index=False).agg({'volumes_PPAM': 'sum'} )
vol_od_calibration.rename(columns={'index_origin': 'origin', 'index_destination': 'destination',"volumes_PPAM":'volumes_calibration'}, inplace = True)

In [46]:
vol_od_calibration.to_csv(output_folder +r'volume_od_calibration.csv')

In [51]:
vol_od_calibration

,origin,destination,volumes_calibration
0,zone_bc_10,zone_bc_10,191.026616
1,zone_bc_10,zone_bc_1000,2.387833
2,zone_bc_10,zone_bc_1004,23.878327
3,zone_bc_10,zone_bc_1011,2.387833
4,zone_bc_10,zone_bc_1018,4.775665
...,...,...,...
672651,zone_bc_999,zone_bc_995,36.046759
672652,zone_bc_999,zone_bc_996,14.418704
672653,zone_bc_999,zone_bc_997,90.116897
672654,zone_bc_999,zone_bc_998,86.512221


# part modale

In [47]:
zones_part_modale = block_name_tract.merge(zones[['index', 'NAME', 'ratio']], on=  'NAME', how = 'left')
zones_part_modale['part_car'] = zones_part_modale['car']/zones_part_modale[['car', 'pt', 'walk']].sum(axis=1) 
zones_part_modale['part_pt'] = zones_part_modale['pt']/zones_part_modale[['car', 'pt', 'walk']].sum(axis=1) 
zones_part_modale['part_walk'] = zones_part_modale['walk']/zones_part_modale[['car', 'pt', 'walk']].sum(axis=1) 

part_calibration = groupby_weighted_average(zones_part_modale, 'index', ['part_car', 'part_pt', 'part_walk'], 'PPAM').rename(columns = {"PPAM":"volumes_calibration"})

In [48]:
part_calibration['PPAM'] = zones_part_modale.groupby(['index'])['PPAM'].sum()
part_calibration.rename(columns = {"PPAM":"volumes_calibration"}, inplace=True)

In [49]:
part_calibration.reset_index().to_csv(output_folder +r'part_modale_calibration.csv')

In [50]:
end_of_notebook

NameError: name 'end_of_notebook' is not defined

# achalandage

In [ ]:
sm = stepmodel.read_zippedpickles(model_folder + 'logit_assignment')
pt_nodes = gpd.read_file(scenario_folder + 'pt/nodes.geojson')
pt_links = gpd.read_file(scenario_folder + 'pt/links.geojson')

loaded_links: 100%|██████████| 38/38 [00:02<00:00, 13.96it/s]   


dallas

In [ ]:
tram = pt_links.loc[pt_links['route_type'] == 'tram'] 
tram_nodes = tram['a'].tolist()
tram_nodes = tram_nodes + (tram['b'].tolist())

nodes = pt_nodes.loc[pt_nodes['index'].isin(tram_nodes)]

In [ ]:
pd.set_option('display.max_row', 359)
comp_boardings_cluster = nodes.merge(sm.node_parenthood['cluster'], left_on= 'index', right_index=True, how='left')

atlanta

In [ ]:
red_line = pt_links.loc[pt_links['route_id'] == 'MARTA_20902']
blue_line = pt_links.loc[pt_links['route_id'] == 'MARTA_20899'] 
green_line = pt_links.loc[pt_links['route_id'] == 'MARTA_20901'] 
subway = pt_links.loc[pt_links['route_type'] == 'subway'] 
tram = pt_links.loc[pt_links['route_type'] == 'tram']

In [ ]:
subway_nodes = subway['a'].tolist()
subway_nodes = subway_nodes + (tram['b'].tolist())

nodes = pt_nodes.loc[pt_nodes['index'].isin(subway_nodes)]
#nodes.to_excel(calib_folder + 'subway_boardings.xlsx')

In [ ]:
comp_boardings_subway = pd.read_excel(calib_folder+'subway_boardings.xlsx')
comp_boardings_cluster = comp_boardings_subway.merge(sm.node_parenthood['cluster'], left_on= 'index', right_index=True, how='left')
#comp_boardings_cluster.to_csv(calib_folder + 'boardings.csv')

houston

In [ ]:
red_line = pt_links.loc[pt_links['route_id'] == 'HOU_700']
green_line = pt_links.loc[pt_links['route_id'] == 'HOU_800'] 
purple_line = pt_links.loc[pt_links['route_id'] == 'HOU_900'] 

In [ ]:
red_line = sm.links.loc[sm.links['route_id'] == 'HOU_700']
green_line = sm.links.loc[sm.links['route_id'] == 'HOU_800'] 
purple_line = sm.links.loc[sm.links['route_id'] == 'HOU_900'] 

In [ ]:
purple_line_nodes=purple_line['a'].tolist()
purple_line_nodes = purple_line_nodes + (purple_line['b'].tolist())

green_line_nodes=green_line['a'].tolist()
green_line_nodes = green_line_nodes + (green_line['b'].tolist())

red_line_nodes=red_line['a'].tolist()
red_line_nodes = red_line_nodes + (red_line['b'].tolist())

In [ ]:

nodes_red = sm.nodes.loc[sm.nodes.index.isin(red_line_nodes)]
nodes_green = sm.nodes.loc[sm.nodes.index.isin(green_line_nodes)]
nodes_purple = sm.nodes.loc[sm.nodes.index.isin(purple_line_nodes)]


In [ ]:

nodes_green.to_excel(calib_folder+'green_line.xlsx')
nodes_red.to_excel(calib_folder+'red_line.xlsx')
nodes_purple.to_excel(calib_folder+'purple_line.xlsx')


In [ ]:
comp_boardings_purple = pd.read_excel(calib_folder+'purple_line.xlsx')
comp_boardings_green = pd.read_excel(calib_folder+'green_line.xlsx')
comp_boardings_red = pd.read_excel(calib_folder+'red_line.xlsx')

In [ ]:
#comp_boardings_red['avg_boardings'] = comp_boardings_red['avg_boardings']/2

In [ ]:
comp_boardings = pd.concat([comp_boardings_red,comp_boardings_green,comp_boardings_purple], ignore_index=True)

In [ ]:
comp_boardings

,index,stop_name,route_id,boardings
0,node_7541,Fannin South Stn NB,HOU_700,1156.5
1,node_7542,Stadium Park / Astrodome NB,HOU_700,328.0
2,node_7543,Stadium Park / Astrodome SB,HOU_700,328.0
3,node_7544,Smith Lands Stn NB,HOU_700,1251.0
4,node_7545,Smith Lands Stn SB,HOU_700,1251.0
...,...,...,...,...
83,node_8204,Theater District Capitol WB,HOU_900,204.5
84,node_8206,Central Station Rusk EB,HOU_900,355.5
85,node_8207,Convention District Rusk EB,HOU_900,95.5
86,node_9963,Theater District Rusk EB,HOU_900,204.5


In [ ]:
comp_boardings.to_csv(calib_folder+'boardings.csv')

In [ ]:
comp_boardings = pd.read_csv(calib_folder+'boardings.csv')
#comp_boardings['avg_boardings_PPAM']= comp_boardings['avg_boardings']*0.25

In [ ]:
comp_boardings_cluster = comp_boardings.merge(sm.node_parenthood['cluster'], left_on= 'index', right_index=True, how='left')

AttributeError: 'StepModel' object has no attribute 'node_parenthood'

bus

In [ ]:
bus_boardings = sm.links.groupby('route_id')['boardings'].sum()
bus_boardings

In [ ]:
bus_boardings.to_excel(calib_folder + 'bus_boardings_model.xlsx')

In [ ]:
bus = pd.read_excel((calib_folder + 'calib_boardings_bus.xlsx'))

In [ ]:
bus = bus[['route_id', 'boardings_model', 'avg_boardings_calib']]
bus.rename(columns={'avg_boardings_calib': 'boardings_calib'}, inplace=True)
bus.to_csv(calib_folder + 'calib_boardings_bus.csv')
bus

In [ ]:
bus = pd.read_csv(calib_folder + 'calib_boardings_bus.csv')

In [ ]:
bus.plot()